# SST Balar analyzer script

In [ ]:
# Import packages
import json
import matplotlib.pyplot as plt
import numpy as np
from common import *
import logging
from yaml import load, dump
from optparse import OptionParser
import yaml
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

In [ ]:
# Load app configuration var
# benchmark_suites_list = ["GPU_Microbenchmark", "rodinia_2.0-ft"]
# benchmark_suites_list = ["rodinia_2.0-ft"]
benchmark_suites_list = ["GPU_Microbenchmark"]
benchmark_suites_table = yaml.load(open("../define-all-apps.yml"), Loader=Loader)
trace_folder = "./hw_traces"
root_logger = logging.getLogger()

# Faulty apps
# Original balar failed to finish
skip_apps = []

# Original balar cannot run these
# skip_apps.extend(["backprop-rodinia-2.0-ft", "heartwall-rodinia-2.0-ft", "lud-rodinia-2.0-ft", "srad_v2-rodinia-2.0-ft", "streamcluster-rodinia-2.0-ft"])

# Orgininal balar has wrong results on these
# skip_apps.extend(["bfs-rodinia-2.0-ft", "hotspot-rodinia-2.0-ft", "nw-rodinia-2.0-ft"])


In [ ]:
# Load stats files
mmio_stats_file = "mmio_stats.json"
original_stats_file = "original_stats.json"
mmio_stats = json.load(open(mmio_stats_file))
original_stats = json.load(open(original_stats_file))


In [ ]:
"""Get the correct bytes ratio for all benchmarks
"""
avg_ratios = []
for app_record in get_benchmark_app(benchmark_suites_list, trace_folder, benchmark_suites_table, root_logger):
    benchmark_suite_name = app_record["benchmark_suite_name"]
    app_name = app_record["app_name"]
    app_args_name = get_argfoldername(app_record["app_args"])

    # Skip some unfinished app in original balar
    if app_name in skip_apps:
        continue
    
    try:
        app_specific_stat = mmio_stats[benchmark_suite_name][app_name][app_args_name]
        sst_stat = app_specific_stat["sst"]
        gpgpusim_stat = app_specific_stat["gpgpusim"]
        # print(sst_stat.keys())
        # print(gpgpusim_stat.keys())
        try:
            ratio_stat = sst_stat["cpu"]["correct_memD2H_ratio"]
            ratio_sum = ratio_stat["Sum.f64"]
            ratio_max = ratio_stat["Max.f64"]
            ratio_min = ratio_stat["Min.f64"]
            ratio_count = ratio_stat["Count.u64"]
            avg_ratio = ratio_sum / ratio_count
            avg_ratios.append(avg_ratio)
            print(f"{app_name}-{app_args_name} correct byte ratio: avg: {avg_ratio} max: {ratio_max} min: {ratio_min}")
        except KeyError:
            root_logger.warning(f"SST TestCPU memcpyD2H ratio stat for app: {app_name} in {benchmark_suite_name} with args: {app_args_name} does not exist!")
    except KeyError:
        root_logger.warning(f"Stats for app: {app_name} in {benchmark_suite_name} with args: {app_args_name} does not exist!")

fig, ax = plt.subplots()
fig.set_dpi(250)
ax.set_title("Average memcpyH2D correct ratio across all benchmarks: \n{}".format("-".join(benchmark_suites_list)))
ax.set_ylabel("Count")
ax.set_xlabel("Correct bytes ratio")
ax.grid(True)
ax.hist(avg_ratios)

In [ ]:
gpgpusim_kernel_stat_keys = ["count", "latency", "dram_traffic", "smem_bk_conflicts", "smem_warp", "gmem_access_generated", "gmem_warp", "exposed_latency", "warp_divergence"]
mmio_gpgpusim_kernel_stats = dict()
original_gpgpusim_kernel_stats = dict()
gpgpusim_kernel_annotations = []
for app_record in get_benchmark_app(benchmark_suites_list, trace_folder, benchmark_suites_table, root_logger):
    benchmark_suite_name = app_record["benchmark_suite_name"]
    app_name = app_record["app_name"]
    app_args_name = get_argfoldername(app_record["app_args"])

    # Skip some unfinished app in original balar
    if app_name in skip_apps:
        continue
    print(app_name)

    errmsg = f"Encount error when processing: {app_name} in {benchmark_suite_name} with args: {app_args_name}"
    try:
        # Get stats for both balar version
        mmio_app_specific_stat = mmio_stats[benchmark_suite_name][app_name][app_args_name]
        original_app_specific_stat = original_stats[benchmark_suite_name][app_name][app_args_name]
        mmio_sst_stat = mmio_app_specific_stat["sst"]
        mmio_gpgpusim_stat = mmio_app_specific_stat["gpgpusim"]
        original_sst_stat = original_app_specific_stat["sst"]
        original_gpgpusim_stat = original_app_specific_stat["gpgpusim"]

        # TODO Analyze gpgpusim kernel stat first
        for kernel in mmio_gpgpusim_stat:
            # try:
            mmio_kernel_stat = mmio_gpgpusim_stat[kernel]
            original_kernel_stat = original_gpgpusim_stat[kernel]
            for line in mmio_kernel_stat:
                    # try:
                mmio_line_stat = mmio_kernel_stat[line]
                original_line_stat = original_kernel_stat[line]
                for i in range(len(gpgpusim_kernel_stat_keys)):
                    key = gpgpusim_kernel_stat_keys[i]
                    mmio_list = mmio_gpgpusim_kernel_stats.setdefault(key, [])
                    original_list = original_gpgpusim_kernel_stats.setdefault(key, [])
                    mmio_list.append(mmio_line_stat[i])
                    original_list.append(original_line_stat[i])
                gpgpusim_kernel_annotations.append(f"{app_name}-{app_args_name}-{kernel}-{line}")
                    # except KeyError as err:
                    #     root_logger.warning(f"Error in access kernel line {line} for {app_name}; msg: {repr(err)}")
                    #     continue
            # except KeyError as err:
            #     root_logger.warning(f"Error in access kernel for {app_name}; msg: {repr(err)}")
            #     continue
    except KeyError as err:
        root_logger.warning(f"{errmsg}; msg: {repr(err)}")

        

In [ ]:
# Plotting the kernel stats
# mmio_gpgpusim_kernel_stats
# original_gpgpusim_kernel_stats
fig, axs = plt.subplots(2, 1)
fig.set_dpi(250)
fig.set_size_inches(10, 10)
fig.suptitle("GPGPUSim kernel stats on benchmarks: \n{}".format("-".join(benchmark_suites_list)))
features = ["count", "latency"]
ax_feature_pair = zip(axs, features)
for (ax, feature) in ax_feature_pair:
    ax.set_title(feature)
    ax.set_ylabel("MMIO")
    ax.set_xlabel("Original")
    ax.grid(True)
    ax.set_xscale("log")
    ax.set_yscale("log")
    original_arr = np.asarray(original_gpgpusim_kernel_stats[feature].copy())
    mmio_arr = np.asarray(mmio_gpgpusim_kernel_stats[feature].copy())
    ax.scatter(original_arr, mmio_arr, c="red", marker="*")
    ax.plot(original_arr, original_arr)

    # Compute mean absolute error over the original balar feature
    mae = (np.abs(original_arr - mmio_arr)).sum() / (original_arr.sum())
    rs = np.corrcoef(original_arr, mmio_arr)
    corcoef = rs[0][1]
    ax.annotate(f"MAE: {mae:.5f}\nr-val: {corcoef :.5f}",
        xy=(0.1, 0.8), xycoords="axes fraction")


    # for i, txt in enumerate(gpgpusim_kernel_annotations):
    #     if original_arr[i] != mmio_arr[i]:
    #         print(txt)